<center>
<h1>Homework #1 - STAT 453</h1>
<h3>By: Jordan Poles</h3>
</center>

In [81]:
from IPython.display import display
from imp import reload
import contingency
from contingency import ContTable
reload(contingency)

<module 'contingency' from '/home/chronos/Development/STAT453/contingency.py'>

## Question 7.2

Using the data from Table 6.7, estimate the Odds Ratio, with an associated 95%
confidence interval, for the association between current exposure to biomass fuel and
tuberculosis. Repeat your analysis using the small sample adjustments for an estimate and
confidence interval for an Odds Ratio. Compare the results from the two approaches and
comment.

In [82]:
biomass = ContTable(50, 238, 21, 524, 1)

2x2 Contingency Table:


,E,~E
D,50,238
~D,21,524


Relative Risk (RR): 4.505621693121693
Odds Ratio (OR): 5.2420968387354945
=> 95% CI using logOR: 1.656722
=> => Upper Bound: 5.277802697128833
=> => Lower Bound: 5.206632540778717
Excess Risk (ER): 0.13507900101936798
Attributable Risk (AR): 0.5479260886419434
ChiSquared Independence Test (n=833)
=> p-value = 3.12705417116e-11


In [83]:
biomass.odds_ratio(CI=.95, verbose=1)

Odds Ratio (OR): 5.2420968387354945
=> 95% CI using logOR: 1.656722
=> => Upper Bound: 5.277802697128833
=> => Lower Bound: 5.206632540778717


(5.2420968387354945,
 1.6567215783542906,
 (1.6499333014540989, 1.6635098552544823))

## Question 7.3

Tuyns et al. (1977) carried out a case-control study of esophageal cancer in the region
known as Ille-et-Vilaine in Brittany, France. The data set, oesoph, can be found at
http://www.crcpress.eom/ejtroducts/downloads/ and is also examined in detail in Breslow
and Day (1980). One risk factor of interest was daily alcohol consumption, measured in
grams per day, given in the data set in four levels: 0 to 39, 40 to 79, 80 to 120, and >120g/day. Dichotomize this risk factor according to whether an individual’s alcohol.
consumption is less than or greater than or equal to 80 g/day. With this binary measure of alcohol consumption, estimate the Odds Ratio with an associated 95% confidence
interval. Also, examine the relationship between incidence of esophageal cancer and the
dichotomized measure of alcohol consumption using the chisq test.

I sourced my dataset from http://forge.scilab.org/index.php/p/rdataset/source/tree/master/csv/datasets/esoph.csv, as the original link was broken.

## Question 9.1


## Question 9.2
